In [1]:
print("Hello!")

Hello!


In [10]:
%%sh 

pip freeze > requirements.txt

In [9]:
%%sh 

pip install pyspark pandas

  Using cached pandas-2.2.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached numpy-2.1.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached numpy-2.1.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.0 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


In [11]:
%%sh 

pip install -r requirements.txt

In [2]:
from os.path import abspath

from pyspark.sql import SparkSession
from pyspark.sql import Row

In [3]:
# warehouse_location points to the default location for managed databases and tables
warehouse_location = abspath('spark-warehouse')

In [4]:
spark = SparkSession \
    .builder \
    .appName("VideoGameEDA") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()

24/09/08 12:53:52 WARN Utils: Your hostname, riddhi-pc resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/09/08 12:53:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/08 12:53:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

24/09/08 12:54:12 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [86]:
csvDF = spark.read.options(header=True).csv('Datasets/Video_Games.csv')

In [22]:
csvDF.show(5)

+--------------------+--------+---------------+------------+---------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+---------+------+
|                Name|Platform|Year_of_Release|       Genre|Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|Critic_Score|Critic_Count|User_Score|User_Count|Developer|Rating|
+--------------------+--------+---------------+------------+---------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+---------+------+
|          Wii Sports|     Wii|           2006|      Sports| Nintendo|   41.36|   28.96|    3.77|       8.45|       82.53|          76|          51|         8|       322| Nintendo|     E|
|   Super Mario Bros.|     NES|           1985|    Platform| Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|        NULL|        NULL|      NULL|      NULL|     NULL|  NULL|
|      Mario Kart Wii|     Wii|           2008|      Racing|

In [93]:
# 1. we can create table using above dataframe
csvDF.write.mode('overwrite').saveAsTable("videogame_eda_tbl")

In [117]:
spark.read.table("videogame_eda_tbl").show(truncate=False)

+--------------------------------------------+--------+---------------+------------+----------------------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+-------------------+------+
|Name                                        |Platform|Year_of_Release|Genre       |Publisher             |NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|Critic_Score|Critic_Count|User_Score|User_Count|Developer          |Rating|
+--------------------------------------------+--------+---------------+------------+----------------------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+-------------------+------+
|Wii Sports                                  |Wii     |2006           |Sports      |Nintendo              |41.36   |28.96   |3.77    |8.45       |82.53       |76          |51          |8         |322       |Nintendo           |E     |
|Super Mario Bros.                           |NES     |1985 

In [92]:
# drop table
spark.sql("DROP TABLE videogame_eda_tbl")

DataFrame[]

In [78]:
# 2. creat table using Hive (It'll be automatically created in Hive Warehouse)
spark.sql("""CREATE TABLE IF NOT EXISTS videogame_eda_tbl
          (Name VARCHAR(50), Platform VARCHAR(10), 
            Year_of_Release INT, Genre VARCHAR(20),
            Publisher VARCHAR(20), NA_Sales FLOAT,
            EU_Sales FLOAT, JP_Sales FLOAT,
            Other_Sales FLOAT, Global_Sales FLOAT,
            Critic_Score FLOAT, Critic_Count INT,
            User_Score FLOAT, User_Count INT,
            Developer VARCHAR(20), Rating VARCHAR(10)
          ) 
          ROW FORMAT DELIMITED 
          FIELDS TERMINATED BY ','
          TBLPROPERTIES ('skip.header.line.count'='1')
          """)

24/09/08 13:43:48 WARN HiveMetaStore: Location: file:/home/riddhipc/Documents/learning/github-projects/videogame-EDA/spark-warehouse/videogame_eda_tbl specified for non-external table:videogame_eda_tbl


DataFrame[]

In [79]:
spark.sql("DESCRIBE TABLE videogame_eda_tbl").show()

+---------------+-----------+-------+
|       col_name|  data_type|comment|
+---------------+-----------+-------+
|           Name|varchar(50)|   NULL|
|       Platform|varchar(10)|   NULL|
|Year_of_Release|        int|   NULL|
|          Genre|varchar(20)|   NULL|
|      Publisher|varchar(20)|   NULL|
|       NA_Sales|      float|   NULL|
|       EU_Sales|      float|   NULL|
|       JP_Sales|      float|   NULL|
|    Other_Sales|      float|   NULL|
|   Global_Sales|      float|   NULL|
|   Critic_Score|      float|   NULL|
|   Critic_Count|        int|   NULL|
|     User_Score|      float|   NULL|
|     User_Count|        int|   NULL|
|      Developer|varchar(20)|   NULL|
|         Rating|varchar(10)|   NULL|
+---------------+-----------+-------+



In [80]:
# check if Table created properlly
spark.sql("""SELECT *
            FROM videogame_eda_tbl""").show()

+----+--------+---------------+-----+---------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+---------+------+
|Name|Platform|Year_of_Release|Genre|Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|Critic_Score|Critic_Count|User_Score|User_Count|Developer|Rating|
+----+--------+---------------+-----+---------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+---------+------+
+----+--------+---------------+-----+---------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+---------+------+



In [81]:
# Load data from .csv file into recently created table (in Hive warehouse)
spark.sql("""LOAD DATA LOCAL INPATH 'Datasets/Video_Games.csv' 
          INTO TABLE videogame_eda_tbl""")


DataFrame[]

In [82]:
# spark.sql("TRUNCATE TABLE videogame_eda_tbl")

In [83]:
spark.sql("SELECT * FROM videogame_eda_tbl").show()

+--------------------+--------+---------------+------------+--------------------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+-------------------+------+
|                Name|Platform|Year_of_Release|       Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|Critic_Score|Critic_Count|User_Score|User_Count|          Developer|Rating|
+--------------------+--------+---------------+------------+--------------------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+-------------------+------+
|                Name|Platform|           NULL|       Genre|           Publisher|    NULL|    NULL|    NULL|       NULL|        NULL|        NULL|        NULL|      NULL|      NULL|          Developer|Rating|
|          Wii Sports|     Wii|           2006|      Sports|            Nintendo|   41.36|   28.96|    3.77|       8.45|       82.53|        76.0|          51|     

In [100]:
help(spark.sql("DESCRIBE FORMATTED videogame_eda_tbl").show(truncate=False))

+----------------------------+-------------+-------+
|col_name                    |data_type    |comment|
+----------------------------+-------------+-------+
|Name                        |string       |NULL   |
|Platform                    |string       |NULL   |
|Year_of_Release             |string       |NULL   |
|Genre                       |string       |NULL   |
|Publisher                   |string       |NULL   |
|NA_Sales                    |string       |NULL   |
|EU_Sales                    |string       |NULL   |
|JP_Sales                    |string       |NULL   |
|Other_Sales                 |string       |NULL   |
|Global_Sales                |string       |NULL   |
|Critic_Score                |string       |NULL   |
|Critic_Count                |string       |NULL   |
|User_Score                  |string       |NULL   |
|User_Count                  |string       |NULL   |
|Developer                   |string       |NULL   |
|Rating                      |string       |NU

In [103]:
# Check files must be store in the  parquet files with snappy compression [Hint - Run Describe command]
# 1. 
spark.sql("DESCRIBE FORMATTED videogame_eda_tbl").show(100,truncate=False)

+----------------------------+------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                             |comment|
+----------------------------+------------------------------------------------------------------------------------------------------+-------+
|Name                        |string                                                                                                |NULL   |
|Platform                    |string                                                                                                |NULL   |
|Year_of_Release             |string                                                                                                |NULL   |
|Genre                       |string                                                                                                |NULL   |
|Publi

In [111]:
%%sh 
# Describe stored file(or table file) informations using following line - 
# 2. using sh command

ls -alh spark-warehouse/videogame_eda_tbl/

total 480K
drwxr-xr-x 2 riddhipc riddhipc 4.0K Sep  8 13:53 .
drwxrwxr-x 3 riddhipc riddhipc 4.0K Sep  8 13:53 ..
-rw-r--r-- 1 riddhipc riddhipc 461K Sep  8 13:53 part-00000-44227652-ac5c-4fd3-bff4-21a5b843014a-c000.snappy.parquet
-rw-r--r-- 1 riddhipc riddhipc 3.7K Sep  8 13:53 .part-00000-44227652-ac5c-4fd3-bff4-21a5b843014a-c000.snappy.parquet.crc
-rw-r--r-- 1 riddhipc riddhipc    0 Sep  8 13:53 _SUCCESS
-rw-r--r-- 1 riddhipc riddhipc    8 Sep  8 13:53 ._SUCCESS.crc


In [123]:
spark.sql("SELECT NAME, COUNT(NAME) FROM videogame_eda_tbl GROUP BY NAME").show(truncate = False)

+-----------------------------------------+-----------+
|NAME                                     |count(NAME)|
+-----------------------------------------+-----------+
|The Elder Scrolls V: Skyrim              |5          |
|The Legend of Zelda: Oracle of Ages      |1          |
|Call of Duty Black Ops: Declassified     |1          |
|Joust                                    |1          |
|Legacy of Kain: Soul Reaver              |1          |
|All-Star Baseball 2003                   |4          |
|J-League Soccer: Prime Goal              |1          |
|RIFT                                     |1          |
|Ninokuni: Shikkoku no Madoushi           |1          |
|Paws & Claws: Dogs & Cats Best Friends   |1          |
|Barnstorming                             |1          |
|Barbie: Jet, Set & Style!                |2          |
|Yu-Gi-Oh! GX: Tag Force 2                |1          |
|Sherlock Holmes: The Mystery of the Mummy|1          |
|ESPN X Games Skateboarding               |2    

In [124]:
spark.sql("SELECT * FROM videogame_eda_tbl \
           WHERE NAME = 'The Elder Scrolls V: Skyrim'").show(truncate=False)

+---------------------------+--------+---------------+------------+------------------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+---------------------+------+
|Name                       |Platform|Year_of_Release|Genre       |Publisher         |NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|Critic_Score|Critic_Count|User_Score|User_Count|Developer            |Rating|
+---------------------------+--------+---------------+------------+------------------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+---------------------+------+
|The Elder Scrolls V: Skyrim|X360    |2011           |Role-Playing|Bethesda Softworks|5.05    |2.79    |0.1     |0.85       |8.79        |96          |89          |8.4       |3589      |Bethesda Game Studios|M     |
|The Elder Scrolls V: Skyrim|PS3     |2011           |Role-Playing|Bethesda Softworks|2.55    |2.6     |0.25    |1          |6.41       

In [137]:
# Calculate the ToTal Sales per Game - North America, EU, JP, Global and Other sales
spark.sql("""SELECT Name, 
          ROUND(SUM(NA_Sales), 2) AS Total_NA_Sales, 
          ROUND(SUM(EU_Sales), 2) AS Total_EU_Sale, 
          ROUND(SUM(JP_Sales), 2) AS Total_JP_Sale, 
          ROUND(SUM(Global_Sales), 2) AS Total_Global_Sale, 
          ROUND(SUM(Other_Sales), 2) AS Total_Other_Sale
            FROM videogame_eda_tbl
            GROUP BY Name
""").show(truncate=False)

+-----------------------------------------+--------------+-------------+-------------+-----------------+----------------+
|Name                                     |Total_NA_Sales|Total_EU_Sale|Total_JP_Sale|Total_Global_Sale|Total_Other_Sale|
+-----------------------------------------+--------------+-------------+-------------+-----------------+----------------+
|The Elder Scrolls V: Skyrim              |9.65          |8.37         |0.39         |21.15            |2.73            |
|The Legend of Zelda: Oracle of Ages      |0.92          |0.53         |0.41         |1.92             |0.06            |
|Call of Duty Black Ops: Declassified     |0.71          |0.43         |0.07         |1.47             |0.26            |
|Joust                                    |1.01          |0.06         |0.0          |1.08             |0.01            |
|Legacy of Kain: Soul Reaver              |0.58          |0.4          |0.0          |1.04             |0.07            |
|All-Star Baseball 2003 

In [153]:
# Replace null values in the "Critic_Score" and "User_Score" columns with appropriate values
# using pyspark - we can't perform delete and update operation in hive table, so we'll update the dataframe
csvDF.na.fill({"Critic_Score" : 0, \
              "User_Score" : 0, \
              "Critic_Count" : 0, \
              "User_Count" : 0, \
              "Developer" : 'N/A', \
              "Rating" : 'N/A'}).show(50, truncate=False)

+--------------------------------------------+--------+---------------+------------+---------------------------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+------------------------------------------------------------+------+
|Name                                        |Platform|Year_of_Release|Genre       |Publisher                  |NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|Critic_Score|Critic_Count|User_Score|User_Count|Developer                                                   |Rating|
+--------------------------------------------+--------+---------------+------------+---------------------------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+------------------------------------------------------------+------+
|Wii Sports                                  |Wii     |2006           |Sports      |Nintendo                   |41.36   |28.96   |3.77    |8.45       |82.53 

In [158]:
# Get total number of the unique Gaming Platforms.
spark.sql("SELECT Platform, COUNT(Platform) AS Total_Platform \
          FROM videogame_eda_tbl \
          GROUP BY Platform").show(100, truncate = False)

+--------+--------------+
|Platform|Total_Platform|
+--------+--------------+
|3DO     |3             |
|PC      |974           |
|PS3     |1331          |
|NES     |98            |
|PS      |1197          |
|DC      |52            |
|GEN     |29            |
|PS2     |2161          |
|3DS     |520           |
|PCFX    |1             |
|GG      |1             |
|WiiU    |147           |
|SNES    |239           |
|GB      |98            |
|SCD     |6             |
|N64     |319           |
|PS4     |393           |
|PSP     |1209          |
|2600    |133           |
|XOne    |247           |
|X360    |1262          |
|GBA     |822           |
|WS      |6             |
|Wii     |1320          |
|GC      |556           |
|PSV     |432           |
|XB      |824           |
|DS      |2152          |
|TG16    |2             |
|NG      |12            |
|SAT     |173           |
+--------+--------------+



In [195]:
# Identify the top-selling games globally and regionally (NA, EU, JP, Globally, Other) - Top 10.
# NA_Sales
spark.sql("""SELECT Name, 
            ROUND(SUM(NA_Sales), 2) AS Top_NA_Sales
            FROM videogame_eda_tbl
            GROUP BY Name
            ORDER BY Top10_NA_Sales DESC
            LIMIT 10""").show(truncate = False)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Top10_NA_Sales` cannot be resolved. Did you mean one of the following? [`Top_NA_Sales`, `spark_catalog`.`default`.`videogame_eda_tbl`.`Name`].; line 5 pos 21;
'GlobalLimit 10
+- 'LocalLimit 10
   +- 'Sort ['Top10_NA_Sales DESC NULLS LAST], true
      +- Aggregate [Name#3023], [Name#3023, round(sum(cast(NA_Sales#3028 as double)), 2) AS Top_NA_Sales#5898]
         +- SubqueryAlias spark_catalog.default.videogame_eda_tbl
            +- Relation spark_catalog.default.videogame_eda_tbl[Name#3023,Platform#3024,Year_of_Release#3025,Genre#3026,Publisher#3027,NA_Sales#3028,EU_Sales#3029,JP_Sales#3030,Other_Sales#3031,Global_Sales#3032,Critic_Score#3033,Critic_Count#3034,User_Score#3035,User_Count#3036,Developer#3037,Rating#3038] parquet


In [194]:
# EU_Sales
spark.sql("""SELECT Name, 
            ROUND(SUM(EU_Sales), 2) AS Top10_EU_Sales
            FROM videogame_eda_tbl
            GROUP BY Name
            ORDER BY Top10_EU_Sales DESC
            LIMIT 10""").show(truncate = False)

+------------------------------+--------------+
|Name                          |Top10_EU_Sales|
+------------------------------+--------------+
|Wii Sports                    |28.96         |
|Grand Theft Auto V            |23.42         |
|Mario Kart Wii                |12.76         |
|FIFA 15                       |12.02         |
|Call of Duty: Modern Warfare 3|11.15         |
|FIFA 16                       |11.09         |
|FIFA 14                       |10.96         |
|Nintendogs                    |10.95         |
|Wii Sports Resort             |10.93         |
|Call of Duty: Black Ops II    |10.84         |
+------------------------------+--------------+



In [193]:
# JP_Sales
spark.sql("""SELECT Name, 
            ROUND(SUM(JP_Sales), 2) AS Top10_JP_Sales
            FROM videogame_eda_tbl
            GROUP BY Name
            ORDER BY Top10_JP_Sales DESC
            LIMIT 10""").show(truncate = False)

+----------------------------------+--------------+
|Name                              |Top10_JP_Sales|
+----------------------------------+--------------+
|Pokemon Red/Pokemon Blue          |10.22         |
|Pokemon Gold/Pokemon Silver       |7.2           |
|Super Mario Bros.                 |6.96          |
|New Super Mario Bros.             |6.5           |
|Pokemon Diamond/Pokemon Pearl     |6.04          |
|Tetris                            |6.03          |
|Pokemon Black/Pokemon White       |5.65          |
|Dragon Quest VII: Warriors of Eden|5.4           |
|Pokemon Ruby/Pokemon Sapphire     |5.38          |
|Animal Crossing: Wild World       |5.33          |
+----------------------------------+--------------+



In [196]:
# Global_Sales
spark.sql("""SELECT Name, 
            ROUND(SUM(Global_Sales), 2) AS Top10_Global_Sales
            FROM videogame_eda_tbl
            GROUP BY Name
            ORDER BY Top10_Global_Sales DESC
            LIMIT 10""").show(truncate = False)

+------------------------------+------------------+
|Name                          |Top10_Global_Sales|
+------------------------------+------------------+
|Wii Sports                    |82.53             |
|Grand Theft Auto V            |56.57             |
|Super Mario Bros.             |45.31             |
|Tetris                        |35.84             |
|Mario Kart Wii                |35.52             |
|Wii Sports Resort             |32.77             |
|Pokemon Red/Pokemon Blue      |31.37             |
|Call of Duty: Black Ops       |30.82             |
|Call of Duty: Modern Warfare 3|30.59             |
|New Super Mario Bros.         |29.8              |
+------------------------------+------------------+



In [197]:
# Other_Sales
spark.sql("""SELECT Name, 
            ROUND(SUM(Other_Sales), 2) AS Top10_Other_Sales
            FROM videogame_eda_tbl
            GROUP BY Name
            ORDER BY Top10_Other_Sales DESC
            LIMIT 10""").show(truncate = False)

+------------------------------+-----------------+
|Name                          |Top10_Other_Sales|
+------------------------------+-----------------+
|Grand Theft Auto: San Andreas |10.71            |
|Wii Sports                    |8.45             |
|Grand Theft Auto V            |7.9              |
|Gran Turismo 4                |7.53             |
|Call of Duty: Black Ops II    |3.76             |
|FIFA Soccer 08                |3.52             |
|Pro Evolution Soccer 2008     |3.51             |
|Call of Duty: Black Ops 3     |3.49             |
|Call of Duty: Black Ops       |3.31             |
|Call of Duty: Modern Warfare 3|3.29             |
+------------------------------+-----------------+



In [216]:
# Popular Genres:Identify which genres are the most popular based on sales.
spark.sql("""SELECT Genre 
            AS Most_Popular_Genre,
            ROUND(SUM(NA_Sales + JP_Sales + EU_Sales + Other_Sales + Global_Sales), 2) 
            AS Total_Sales
            FROM videogame_eda_tbl
            WHERE Genre IS NOT NULL 
            GROUP BY Genre
            ORDER BY Total_Sales DESC""").show(100, truncate = False)

+------------------+-----------+
|Most_Popular_Genre|Total_Sales|
+------------------+-----------+
|Action            |3489.45    |
|Sports            |2663.27    |
|Shooter           |2105.39    |
|Role-Playing      |1869.01    |
|Platform          |1655.85    |
|Misc              |1605.69    |
|Racing            |1457.57    |
|Fighting          |895.01     |
|Simulation        |780.68     |
|Puzzle            |485.59     |
|Adventure         |475.28     |
|Strategy          |348.73     |
+------------------+-----------+



In [228]:
# Find total users per gaming platform
spark.sql("""SELECT Platform, 
          SUM(User_Count) AS Total_Users
          FROM videogame_eda_tbl
          GROUP BY Platform
          ORDER BY Total_Users DESC
          """).show(100, truncate = False)

+--------+-----------+
|Platform|Total_Users|
+--------+-----------+
|PC      |473781.0   |
|X360    |180793.0   |
|PS3     |180567.0   |
|PS4     |133166.0   |
|PS2     |54186.0    |
|XOne    |52010.0    |
|Wii     |29749.0    |
|WiiU    |22381.0    |
|3DS     |16041.0    |
|DS      |15764.0    |
|XB      |14216.0    |
|PSV     |13776.0    |
|GC      |13084.0    |
|PSP     |12808.0    |
|PS      |11964.0    |
|GBA     |5856.0     |
|DC      |1183.0     |
|3DO     |NULL       |
|NES     |NULL       |
|GEN     |NULL       |
|PCFX    |NULL       |
|GG      |NULL       |
|SNES    |NULL       |
|GB      |NULL       |
|SCD     |NULL       |
|N64     |NULL       |
|2600    |NULL       |
|WS      |NULL       |
|TG16    |NULL       |
|NG      |NULL       |
|SAT     |NULL       |
+--------+-----------+

